In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [13]:
subjects_info = pd.read_csv('/kaggle/input/open-nuro-dataset/dataset/participants.tsv', delimiter='\t')
from sklearn.model_selection import train_test_split
import pandas as pd

# First, let's identify the unique groups in the DataFrame
groups = subjects_info['Group'].unique()
groups=["A","C"]
# Now, let's split each group individually
train_dfs = []
test_dfs = []
total=[]
for group in groups:
    # Filter the DataFrame for the current group
    group_df = subjects_info[subjects_info['Group'] == group]
    
    # Split the group data into training and testing sets while maintaining balance in gender
    train_group, test_group = train_test_split(group_df, test_size=0.3, stratify=group_df['Gender'], random_state=42)
    total.append(group_df)
    # Append the split data to the lists
    train_dfs.append(train_group)
    test_dfs.append(test_group)

# Concatenate the training and testing DataFrames for all groups
train_df = pd.concat(train_dfs)
test_df = pd.concat(test_dfs)
total_df=pd.concat(total)
# Now, train_df and test_df contain the split data with balanced groups and secondary balance in gender
# Extracting subject IDs from the training set
training_subjects = train_df['participant_id'].str.extract(r'sub-(\d+)').astype(int).squeeze().unique().tolist()

# Extracting subject IDs from the testing set
testing_subjects = test_df['participant_id'].str.extract(r'sub-(\d+)').astype(int).squeeze().unique().tolist()

total_subjects = total_df['participant_id'].str.extract(r'sub-(\d+)').astype(int).squeeze().unique().tolist()

# Displaying the lists of subjects
print("Training Subjects:")
print(training_subjects)

print("\nTesting Subjects:")
print(testing_subjects)

print("\nTotal Subjects:")
print(total_subjects)

Training Subjects:
[17, 18, 10, 35, 6, 2, 5, 4, 27, 9, 3, 14, 30, 31, 29, 15, 34, 25, 8, 22, 1, 23, 24, 11, 7, 58, 41, 44, 51, 50, 49, 45, 37, 53, 65, 60, 38, 61, 62, 57, 64, 54, 59, 55, 46]

Testing Subjects:
[28, 12, 26, 21, 36, 19, 33, 32, 16, 20, 13, 52, 63, 48, 43, 47, 39, 40, 56, 42]

Total Subjects:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65]


In [14]:
##laods the data

import numpy as np
from scipy.io import loadmat
import numpy as np
from sklearn.preprocessing import OneHotEncoder

def load_data(subjects_info,subjects, data_type='training'):
    groups=[]
    morlet = []
    rbps = []
    scc=[]
    labels = []
    for idx in subjects:
        file_path_morlet = f"/kaggle/input/time-freq-transforms/cleaned_data_og/morlet_transform/subject_{idx}.npy"
        subject_data_morlet = np.load(file_path_morlet)
        
        morlet.append(subject_data_morlet)
        
#         file_path_rbp = f"/kaggle/input/extraction-cleaned-custom-cleaned/rbp/rbp_{idx}.npy"
#         subject_rbp = np.load(file_path_rbp)
        
#         rbps.append(subject_rbp)
        
#         file_path_scc = f"/kaggle/input/extraction-cleaned-custom-cleaned/scc_cleaned_base/subject_{idx}_scc.npy"
#         subject_scc = np.load(file_path_scc)
        
#         scc.append(subject_scc)
        
        # Duplicate entries in the 'Group' column
        num_epochs = subject_data_morlet.shape[0]  # Number of epochs, replace 'your_data_key' as before
        subject_id = f"sub-{str(idx).zfill(3)}"  # Format subject ID
        group_info = subjects_info[subjects_info['participant_id'] == subject_id]['Group'].values[0]
        duplicated_groups = [group_info] * num_epochs
        labels.extend(duplicated_groups)
        
        groups.extend([idx] * num_epochs)
        
    return np.concatenate(morlet, axis=0),labels,groups

In [15]:
###source file
subjects_info = pd.read_csv('/kaggle/input/open-nuro-dataset/dataset/participants.tsv', delimiter='\t')
##########

In [ ]:
morlet_total, total_labels, groups_total = load_data(subjects_info, total_subjects, data_type='total')

In [ ]:
# Assuming you define label_mapping, encoder, and encode labels as in your code
#####################################################
label_mapping = {'A': 0, 'C': 1, 'F': 2}  # You can adjust this mapping as needed
#####################################################
df=pd.Series(total_labels).map(label_mapping)

encoder = OneHotEncoder(categories='auto', sparse=False)
encoder.fit(np.array(pd.Series(total_labels).map(label_mapping)).reshape(-1, 1))
# Encode labels
################################################

total_labels_encoded = encoder.transform(np.array(pd.Series(total_labels).map(label_mapping).values).reshape(-1, 1))
################################################


print("Total labels shape:", total_labels_encoded.shape)

In [10]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
#####total dataset 
morlet_tensor_total = torch.tensor(morlet_total, dtype=torch.float32)
labels_tensor_total = torch.tensor(total_labels_encoded, dtype=torch.float32)

dataset_total = TensorDataset(morlet_tensor_total, labels_tensor_total)

batch_size = 64

train_loader = DataLoader(dataset_total, batch_size=batch_size, shuffle=True,drop_last=True)

In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import classification_report

# Step 1: Define the modified ResNet model
class ModifiedResNet(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedResNet, self).__init__()
        # Load the pre-trained ResNet model
        resnet = models.resnet50(pretrained=True)
        
        # Modify the input layer
        resnet.conv1 = nn.Conv2d(19, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        # Modify the output layer
        resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)
        
        # Freeze the parameters of the pre-trained layers
        for param in resnet.parameters():
            param.requires_grad = True
        
        # Make the parameters of the modified layers trainable
        for param in resnet.fc.parameters():
            param.requires_grad = True
        
        self.resnet = resnet

    def forward(self, x):
        return self.resnet(x)



In [11]:
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score

# Load your dataset

# Assuming your dataset is in the form of a list of dictionaries
# where each dictionary represents a subject with data, labels, and a group identifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 256
logo = LeaveOneGroupOut()
group_indices = logo.split(np.arange(len(dataset_total)), groups=groups_total)
total_y_true = []
total_y_pred = []

for train_indices, test_indices in group_indices:
    # Split the dataset into train and test sets based on group indices
    train_data = [dataset_total[i] for i in train_indices]
    test_data = [dataset_total[i] for i in test_indices]

    # Create DataLoader for train and test sets
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    # Initialize the model and optimizer
    model = ModifiedResNet(2).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Early stopping parameters
    patience = 2# Number of epochs to wait before early stopping
    best_val_loss = float('inf')
    early_stop_counter = 0
    best_model_state = None  # Variable to store the best model's state

    # Training loop
    num_epochs = 150
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs1, labels in train_loader:
            optimizer.zero_grad()
            inputs1 = inputs1.to(device)
            labels = labels.to(device)
            outputs = model(inputs1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Calculate validation loss
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs1, labels in test_loader:
                inputs1 = inputs1.to(device)
                labels = labels.to(device)
                outputs = model(inputs1)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        val_loss /= len(test_loader)
        print(f"Epoch {epoch+1}, Train Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss}")

        # Early stopping and save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()  # Save the best model's state
            early_stop_counter = 0
        else:
            early_stop_counter += 1

        if early_stop_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    # Load the best model's state
    model.load_state_dict(best_model_state)

    # Evaluate on the test set
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs1, labels in test_loader:
            inputs1 = inputs1.to(device)
            labels = labels.to(device)
            outputs = model(inputs1)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().tolist())
            y_pred.extend(predicted.cpu().tolist())

    total_y_true.extend(labels.cpu().tolist())
    total_y_pred.extend(predicted.cpu().tolist())

# Calculate classification report
report = classification_report(np.argmax(y_true, axis=1), y_pred)
print("Classification Report:")
print(report)
print("\n")

report = classification_report(np.argmax(total_y_true, axis=1), total_y_pred)
print("total_Classification Report:")
print(report)
print("\n")

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Train Loss: 0.7315985219819205, Val Loss: 6.187893867492676
Epoch 2, Train Loss: 3.4689119287899564, Val Loss: 0.2843853235244751
Epoch 3, Train Loss: 0.7141327857971191, Val Loss: 1.0053882598876953
Epoch 4, Train Loss: 0.7050550920622689, Val Loss: 0.6610081791877747
Early stopping at epoch 4
Epoch 1, Train Loss: 0.7091161438396999, Val Loss: 0.09184339642524719
Epoch 2, Train Loss: 4.278968410832541, Val Loss: 1.3558499813079834
Epoch 3, Train Loss: 0.98891282081604, Val Loss: 0.44086745381355286
Early stopping at epoch 3
Epoch 1, Train Loss: 0.723374034677233, Val Loss: 15.476301193237305
Epoch 2, Train Loss: 20.18958272252764, Val Loss: 0.7087131142616272
Epoch 3, Train Loss: 0.9738549760409764, Val Loss: 0.731797993183136
Epoch 4, Train Loss: 0.7014114090374538, Val Loss: 0.6141326427459717
Epoch 5, Train Loss: 0.7049615383148193, Val Loss: 0.6544997096061707
Epoch 6, Train Loss: 0.6917514375277928, Val Loss: 0.5927849411964417
Epoch 7, Train Loss: 0.6910059026309422, Va

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.

In [18]:
from sklearn.metrics import classification_report

report = classification_report(np.argmax(total_y_true, axis=1), total_y_pred)
print("total_Classification Report:")
print(report)
print("\n")

AxisError: axis 1 is out of bounds for array of dimension 1

In [8]:
import torchvision.models as models

resnext = models.resnext101_32x8d(pretrained=True)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt101_32X8D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt101_32X8D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /root/.cache/torch/hub/checkpoints/resnext101_32x8d-8ba56ff5.pth
100%|██████████| 340M/340M [00:06<00:00, 57.6MB/s] 


In [13]:
class ModifiedResNeXt(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedResNeXt, self).__init__()
        # Load the pre-trained ResNeXt model
        resnext = models.resnext101_32x8d(pretrained=True)

        # Modify the input layer
        resnext.conv1 = nn.Conv2d(19, 64, kernel_size=7, stride=2, padding=3, bias=False)

        # Modify the output layer
        resnext.fc = nn.Linear(resnext.fc.in_features, num_classes)

        # Freeze the parameters of the pre-trained layers
        for param in resnext.parameters():
            param.requires_grad = True

        # Make the parameters of the modified layers trainable
        for param in resnext.fc.parameters():
            param.requires_grad = True

        self.resnext = resnext

    def forward(self, x):
        return self.resnext(x)

In [15]:
torch.cuda.empty_cache()

In [16]:
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score

# Load your dataset

# Assuming your dataset is in the form of a list of dictionaries
# where each dictionary represents a subject with data, labels, and a group identifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 256
logo = LeaveOneGroupOut()
group_indices = logo.split(np.arange(len(dataset_total)), groups=groups_total)
total_y_true = []
total_y_pred = []

for train_indices, test_indices in group_indices:
    # Split the dataset into train and test sets based on group indices
    train_data = [dataset_total[i] for i in train_indices]
    test_data = [dataset_total[i] for i in test_indices]

    # Create DataLoader for train and test sets
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    # Initialize the model and optimizer
    model = ModifiedResNeXt(2).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Early stopping parameters
    patience = 2# Number of epochs to wait before early stopping
    best_val_loss = float('inf')
    early_stop_counter = 0
    best_model_state = None  # Variable to store the best model's state

    # Training loop
    num_epochs = 150
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs1, labels in train_loader:
            optimizer.zero_grad()
            inputs1 = inputs1.to(device)
            labels = labels.to(device)
            outputs = model(inputs1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Calculate validation loss
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs1, labels in test_loader:
                inputs1 = inputs1.to(device)
                labels = labels.to(device)
                outputs = model(inputs1)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        val_loss /= len(test_loader)
        print(f"Epoch {epoch+1}, Train Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss}")

        # Early stopping and save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()  # Save the best model's state
            early_stop_counter = 0
        else:
            early_stop_counter += 1

        if early_stop_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    # Load the best model's state
    model.load_state_dict(best_model_state)

    # Evaluate on the test set
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs1, labels in test_loader:
            inputs1 = inputs1.to(device)
            labels = labels.to(device)
            outputs = model(inputs1)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().tolist())
            y_pred.extend(predicted.cpu().tolist())

    total_y_true.extend(labels.cpu().tolist())
    total_y_pred.extend(predicted.cpu().tolist())

# Calculate classification report
report = classification_report(np.argmax(y_true, axis=1), y_pred)
print("Classification Report:")
print(report)
print("\n")

report = classification_report(np.argmax(total_y_true, axis=1), total_y_pred)
print("total_Classification Report:")
print(report)
print("\n")

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt101_32X8D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt101_32X8D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 15.89 GiB of which 34.12 MiB is free. Process 2451 has 15.86 GiB memory in use. Of the allocated memory 15.57 GiB is allocated by PyTorch, and 888.00 KiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [7]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import classification_report

# Step 1: Define the modified Inception v3 model
class ModifiedInceptionV3(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedInceptionV3, self).__init__()
        # Load the pre-trained Inception v3 model
        inception = models.inception_v3(pretrained=True)

        # Modify the input layer
        inception.Conv2d_1a_3x3.conv = nn.Conv2d(19, 32, kernel_size=3, stride=2, bias=False)

        # Modify the output layer
        num_ftrs = inception.fc.in_features
        inception.fc = nn.Linear(num_ftrs, num_classes)

        # Freeze the parameters of the pre-trained layers
        for param in inception.parameters():
            param.requires_grad = True

        # Make the parameters of the modified layers trainable
        for param in inception.fc.parameters():
            param.requires_grad = True

        self.inception = inception

    def forward(self, x):
        return self.inception(x)



In [11]:
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score

# Load your dataset

# Assuming your dataset is in the form of a list of dictionaries
# where each dictionary represents a subject with data, labels, and a group identifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 256
logo = LeaveOneGroupOut()
group_indices = logo.split(np.arange(len(dataset_total)), groups=groups_total)
total_y_true = []
total_y_pred = []

for train_indices, test_indices in group_indices:
    # Split the dataset into train and test sets based on group indices
    train_data = [dataset_total[i] for i in train_indices]
    test_data = [dataset_total[i] for i in test_indices]

    # Create DataLoader for train and test sets
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    # Initialize the model and optimizer
    model = ModifiedInceptionV3(2).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Early stopping parameters
    patience = 2# Number of epochs to wait before early stopping
    best_val_loss = float('inf')
    early_stop_counter = 0
    best_model_state = None  # Variable to store the best model's state

    # Training loop
    num_epochs = 150
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs1, labels in train_loader:
            optimizer.zero_grad()
            inputs1 = inputs1.to(device)
            labels = labels.to(device)
            outputs = model(inputs1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Calculate validation loss
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs1, labels in test_loader:
                inputs1 = inputs1.to(device)
                labels = labels.to(device)
                outputs = model(inputs1)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        val_loss /= len(test_loader)
        print(f"Epoch {epoch+1}, Train Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss}")

        # Early stopping and save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()  # Save the best model's state
            early_stop_counter = 0
        else:
            early_stop_counter += 1

        if early_stop_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    # Load the best model's state
    model.load_state_dict(best_model_state)

    # Evaluate on the test set
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs1, labels in test_loader:
            inputs1 = inputs1.to(device)
            labels = labels.to(device)
            outputs = model(inputs1)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().tolist())
            y_pred.extend(predicted.cpu().tolist())

    total_y_true.extend(labels.cpu().tolist())
    total_y_pred.extend(predicted.cpu().tolist())

# Calculate classification report
report = classification_report(np.argmax(y_true, axis=1), y_pred)
print("Classification Report:")
print(report)
print("\n")

report = classification_report(np.argmax(total_y_true, axis=1), total_y_pred)
print("total_Classification Report:")
print(report)
print("\n")

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 149MB/s]  


RuntimeError: Given groups=1, weight of size [32, 19, 3, 3], expected input[256, 3, 27, 750] to have 19 channels, but got 3 channels instead

In [12]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Generate synthetic data
X, y = make_classification(n_samples=1735, n_features=20, n_classes=2, n_clusters_per_class=1, random_state=65)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a logistic regression classifier
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = classifier.predict(X_test)

# Generate the classification report
report = classification_report(y_test, y_pred)

print("Classification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       261
           1       1.00      0.98      0.99       260

    accuracy                           0.99       521
   macro avg       0.99      0.99      0.99       521
weighted avg       0.99      0.99      0.99       521



In [24]:
from sklearn.metrics import classification_report

# Generate some example data
y_true = [0] * 948 + [1] * 787  # 0 represents class 1, 1 represents class 2
y_pred = [0] * 250 + [1] * 1485  # Predicting class 1 for 200 points, class 2 for 1535 points

# Print the classification report
print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.26      0.42       948
           1       0.53      1.00      0.69       787

    accuracy                           0.60      1735
   macro avg       0.76      0.63      0.56      1735
weighted avg       0.79      0.60      0.54      1735

